Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [4]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [5]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv',
                             na_values=[0, -2.000000e-08],
                            parse_dates=['date_recorded']), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv')).set_index('id')

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv',
                   parse_dates=['date_recorded'],
                   na_values=[0, -2.000000e-08])
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

# WRANGLE

In [6]:
def wrangle(X):

  #Make a copy
  X = X.copy()

  #Create yr_recorded column
  X['yr_recorded'] = X['date_recorded'].dt.year

  ## Feature Engineering
  X['pump_age'] = X['yr_recorded'] - X['construction_year']
  X['tsh_per_person'] = X['amount_tsh'] / X['population']
  X['tsh_by_height'] = X['amount_tsh'] / X['gps_height']

  #Column list to remove
  columns=['date_recorded', 'recorded_by', 'quality_group', 'extraction_type_group']
  X = X.drop(columns=columns)

  return X

In [7]:
train = wrangle(train)
X_test = wrangle(test)

# SPLIT

NO NEED TO SPLIT, USING CV

In [8]:
target = 'status_group'
y = train[target]
X = train.drop(columns=target)

# BASELINE

In [9]:
print('Baseline Accuracy:', y.value_counts(normalize=True).max())

Baseline Accuracy: 0.5430899510092763


# BUILD MODEL

In [10]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [11]:
rf = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_jobs=-1)
)

In [12]:
rf_scores = cross_val_score(rf, X, y, cv=5)

In [13]:
rf_scores.mean()

0.8085825919208357

# TUNE

In [14]:
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [72]:
param_grid = {
    'randomforestclassifier__n_estimators': range(25, 251, 25),
    'randomforestclassifier__max_depth': range(25, 101, 25)
}

rf_rs = RandomizedSearchCV(
    rf,
    param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1
)

In [73]:
rf_rs.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/home/dakotagporter/anaconda3/envs/unit2/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  7.3min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('ordinalencoder',
                                              OrdinalEncoder()),
                                             ('simpleimputer', SimpleImputer()),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(n_jobs=-1,
                                                                     random_state=42))]),
                   n_jobs=-1,
                   param_distributions={'randomforestclassifier__max_depth': range(25, 101, 25),
                                        'randomforestclassifier__n_estimators': range(25, 251, 25)},
                   verbose=1)

In [74]:
rf_rs.best_params_ ## 250, 0.6, 100

{'randomforestclassifier__n_estimators': 175,
 'randomforestclassifier__max_depth': 25}

In [26]:
#rf_gs = GridSearchCV(
    rf,
    param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1
)

In [27]:
#rf_gs.fit(X, y)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.9min
/home/dakotagporter/anaconda3/envs/unit2/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 42.2min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 77.9min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed: 78.7min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                                       ('simpleimputer', SimpleImputer()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(n_jobs=-1,
                                                               random_state=42))]),
             n_jobs=-1,
             param_grid={'randomforestclassifier__max_depth': range(25, 101, 25),
                         'randomforestclassifier__max_samples': array([0.2, 0.4, 0.6, 0.8]),
                         'randomforestclassifier__n_estimators': range(25, 251, 25)},
             verbose=1)

In [28]:
rf_gs.best_params_ #50, 0.6, 225

{'randomforestclassifier__max_depth': 50,
 'randomforestclassifier__max_samples': 0.6000000000000001,
 'randomforestclassifier__n_estimators': 225}

In [41]:
from skopt import BayesSearchCV

opt = BayesSearchCV(rf, param_grid)

ValueError: Dimension has to be a list or tuple.

# NEW MODEL/CHECK METRICS

In [81]:
rf_tuned = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(
        n_estimators=175,
        max_depth=25,
        max_samples=None,
        random_state=4,
        n_jobs=-1)
)

In [82]:
rf_tuned_scores = cross_val_score(rf_tuned, X, y, cv=5)

In [83]:
rf_tuned_scores.mean() # 0.812202

0.8105860524599476

# SUBMISSION

In [84]:
y_pred = rf_tuned.predict(X_test)

submission = pd.DataFrame({'id': X_test.index, 'status_group': y_pred})
submission.head()

ValueError: Must train encoder before it can be used to transform data.

In [ ]:
time = pd.Timestamp.now().strftime('%Y-%m-%d-%H-%M_')
submission.to_csv(f'{time}rf_submission.csv', index=False)